In [1]:
import os # hablar con el sistema operativo
import glob # listar directorios y ese tipo de operaciones
import itertools # herramientas para iterar objetos
from pathlib import Path # manipular rutas a directorios
import zipfile # comprimir y descomprimir archivos
import numpy as np # operaciones vectorizadas
import pandas as pd # DataFrames
from datetime import timedelta, date, datetime # Manejar fechas
import openpyxl # leer/escribir archivos de exel
import requests # Hablar con direcciones web
import logging

## Leer datos

In [ ]:
df = pd.read_pickle("data/datos_covid_ene19_todos.pkl")

## Filtrar

* Menores (iguales) a 19 años (pacientes pediátricos)
* Sólo casos confirmados

In [6]:
df['CLASIFICACION_FINAL'].unique()

array(['CASO SOSPECHOSO', 'NEGATIVO A SARS-COV-2',
       'CASO DE SARS-COV-2  CONFIRMADO', 'NO REALIZADO POR LABORATORIO',
       'CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA',
       'CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN',
       'INVÁLIDO POR LABORATORIO'], dtype=object)

In [7]:
valores_confirmados = ['CASO DE SARS-COV-2  CONFIRMADO', 
                       'CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA',
                       'CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN']
df = df.query('EDAD <= 19')
df = df.loc[df["CLASIFICACION_FINAL"].isin(valores_confirmados)]
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,TOMA_MUESTRA_ANTIGENO_BIN,MIGRANTE_BIN,UCI_BIN,DEFUNCION,AÑO_INGRESO,MES_INGRESO,DIA_SEMANA_INGRESO,SEMANA_AÑO_INGRESO,DIA_MES_INGRESO,DIA_AÑO_INGRESO
FECHA_SINTOMAS,,,,,,,,,,,,,,,,,,,,,
2020-06-26,2022-01-18,z388cd,1,12,TLAXCALA,2,MÉXICO,PUEBLA,RAFAEL LARA GRAJALES,HOSPITALIZADO,...,0,0,0,0,2020,6,4,26,26,178
2020-05-26,2022-01-18,027d77,1,12,TLAXCALA,2,PUEBLA,PUEBLA,NOPALUCAN,HOSPITALIZADO,...,0,0,0,0,2020,5,1,22,26,147
2020-03-01,2022-01-18,1d457b,1,6,GUANAJUATO,2,GUANAJUATO,GUANAJUATO,LEÓN,AMBULATORIO,...,0,0,0,0,2020,3,6,9,1,61
2020-03-02,2022-01-18,143f8c,1,6,GUANAJUATO,1,GUANAJUATO,GUANAJUATO,LEÓN,AMBULATORIO,...,0,0,0,0,2020,3,0,10,2,62
2020-03-02,2022-01-18,07ce20,1,12,ZACATECAS,1,ZACATECAS,ZACATECAS,OJOCALIENTE,AMBULATORIO,...,0,0,0,0,2020,3,0,10,2,62


## Grupos de edad

In [13]:
bins = [0, 2, 4, 9, 14, 19]
df['Grupo de Edad'] = pd.cut(df['EDAD'], bins, labels=['Menor de 2 años', '2 a 4 años', 
                                 '5 a 9 años', '10 a 14 años', 
                                 '15 a 19 años'])
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,MIGRANTE_BIN,UCI_BIN,DEFUNCION,AÑO_INGRESO,MES_INGRESO,DIA_SEMANA_INGRESO,SEMANA_AÑO_INGRESO,DIA_MES_INGRESO,DIA_AÑO_INGRESO,Grupo de Edad
FECHA_SINTOMAS,,,,,,,,,,,,,,,,,,,,,
2020-06-26,2022-01-18,z388cd,1,12,TLAXCALA,2,MÉXICO,PUEBLA,RAFAEL LARA GRAJALES,HOSPITALIZADO,...,0,0,0,2020,6,4,26,26,178,10 a 14 años
2020-05-26,2022-01-18,027d77,1,12,TLAXCALA,2,PUEBLA,PUEBLA,NOPALUCAN,HOSPITALIZADO,...,0,0,0,2020,5,1,22,26,147,Menor de 2 años
2020-03-01,2022-01-18,1d457b,1,6,GUANAJUATO,2,GUANAJUATO,GUANAJUATO,LEÓN,AMBULATORIO,...,0,0,0,2020,3,6,9,1,61,5 a 9 años
2020-03-02,2022-01-18,143f8c,1,6,GUANAJUATO,1,GUANAJUATO,GUANAJUATO,LEÓN,AMBULATORIO,...,0,0,0,2020,3,0,10,2,62,5 a 9 años
2020-03-02,2022-01-18,07ce20,1,12,ZACATECAS,1,ZACATECAS,ZACATECAS,OJOCALIENTE,AMBULATORIO,...,0,0,0,2020,3,0,10,2,62,10 a 14 años


## Columnas Hombre - Mujer

In [58]:
df = pd.concat([df, pd.get_dummies(df['SEXO']).rename(columns={'1':'MUJER', '2':'HOMBRE'})], axis=1)

## Columnas Ambulatorio - Hospitalizado

In [60]:
df = pd.concat([df, pd.get_dummies(df['TIPO_PACIENTE'])], axis=1)
df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,MES_INGRESO,DIA_SEMANA_INGRESO,SEMANA_AÑO_INGRESO,DIA_MES_INGRESO,DIA_AÑO_INGRESO,Grupo de Edad,MUJER,HOMBRE,AMBULATORIO,HOSPITALIZADO
FECHA_SINTOMAS,,,,,,,,,,,,,,,,,,,,,
2020-06-26,2022-01-18,z388cd,1,12,TLAXCALA,2,MÉXICO,PUEBLA,RAFAEL LARA GRAJALES,HOSPITALIZADO,...,6,4,26,26,178,10 a 14 años,0,1,0,1
2020-05-26,2022-01-18,027d77,1,12,TLAXCALA,2,PUEBLA,PUEBLA,NOPALUCAN,HOSPITALIZADO,...,5,1,22,26,147,Menor de 2 años,0,1,0,1
2020-03-01,2022-01-18,1d457b,1,6,GUANAJUATO,2,GUANAJUATO,GUANAJUATO,LEÓN,AMBULATORIO,...,3,6,9,1,61,5 a 9 años,0,1,1,0
2020-03-02,2022-01-18,143f8c,1,6,GUANAJUATO,1,GUANAJUATO,GUANAJUATO,LEÓN,AMBULATORIO,...,3,0,10,2,62,5 a 9 años,1,0,1,0
2020-03-02,2022-01-18,07ce20,1,12,ZACATECAS,1,ZACATECAS,ZACATECAS,OJOCALIENTE,AMBULATORIO,...,3,0,10,2,62,10 a 14 años,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-15,2022-01-18,m02eef0,2,12,MÉXICO,2,MÉXICO,MÉXICO,NO ESPECIFICADO,AMBULATORIO,...,10,4,41,15,288,15 a 19 años,0,1,1,0
2021-10-15,2022-01-18,m0f5e7f,2,12,MÉXICO,2,MÉXICO,MÉXICO,NO ESPECIFICADO,AMBULATORIO,...,10,4,41,15,288,15 a 19 años,0,1,1,0
2021-10-15,2022-01-18,m137c6f,2,12,MÉXICO,2,MÉXICO,MÉXICO,NO ESPECIFICADO,AMBULATORIO,...,10,4,41,15,288,10 a 14 años,0,1,1,0


## Agregados

In [105]:
agregados =   (df.groupby([pd.Grouper(freq='W'), "ENTIDAD_RES", 'Grupo de Edad'])
               .agg({'ID_REGISTRO':'size',
                     'MUJER':'sum',
                     'HOMBRE':'sum',
                     'DEFUNCION':'sum',
                     'AMBULATORIO': 'sum',
                     'HOSPITALIZADO': 'sum',
                     'UCI_BIN':'sum',
                     'INTUBADO_BIN':'sum',
                     'DIABETES_BIN':'sum',
                     'EPOC_BIN': 'sum',
                     'ASMA_BIN': 'sum',
                     'INMUSUPR_BIN':'sum',
                     'HIPERTENSION_BIN':'sum',
                     'CARDIOVASCULAR_BIN':'sum',
                     'OBESIDAD_BIN': 'sum',
                     'RENAL_CRONICA_BIN': 'sum',
                     'TABAQUISMO_BIN':'sum',
                     'OTRAS_COM_BIN':'sum'        
                     })
                .rename(columns={'ID_REGISTRO':'Casos Confirmados'}))
agregados.head()

Casos Confirmados  MUJER  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                               
2020-03-01     AGUASCALIENTES Menor de 2 años                  0    0.0   
                              2 a 4 años                       0    0.0   
                              5 a 9 años                       0    0.0   
                              10 a 14 años                     0    0.0   
                              15 a 19 años                     0    0.0   

                                               HOMBRE  DEFUNCION  AMBULATORIO  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                                     
2020-03-01     AGUASCALIENTES Menor de 2 años     0.0          0          0.0   
                              2 a 4 años          0.0          0          0.0   
                              5 a 9 años          0.0          0          0.0   
                              10 a 14 años        0.0          0          0.0   
                              15 a 19 años        0.0          0          0.0   

                                               HOSPITALIZADO  UCI_BIN  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                             
2020-03-01     AGUASCALIENTES Menor de 2 años              0        0   
                              2 a 4 años                   0        0   
                              5 a 9 años                   0        0   
                              10 a 14 años                 0        0   
                              15 a 19 años                 0        0   

                                               INTUBADO_BIN  DIABETES_BIN  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                                 
2020-03-01     AGUASCALIENTES Menor de 2 años             0             0   
                              2 a 4 años                  0             0   
                              5 a 9 años                  0             0   
                              10 a 14 años                0             0   
                              15 a 19 años                0             0   

                                               EPOC_BIN  ASMA_BIN  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                         
2020-03-01     AGUASCALIENTES Menor de 2 años         0         0   
                              2 a 4 años              0         0   
                              5 a 9 años              0         0   
                              10 a 14 años            0         0   
                              15 a 19 años            0         0   

                                               INMUSUPR_BIN  HIPERTENSION_BIN  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                                     
2020-03-01     AGUASCALIENTES Menor de 2 años             0                 0   
                              2 a 4 años                  0                 0   
                              5 a 9 años                  0                 0   
                              10 a 14 años                0                 0   
                              15 a 19 años                0                 0   

                                               CARDIOVASCULAR_BIN  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                         
2020-03-01     AGUASCALIENTES Menor de 2 años                   0   
                              2 a 4 años                        0   
                              5 a 9 años                        0   
                              10 a 14 años                      0   
                              15 a 19 años                      0   

                                               OBESIDAD_BIN  \
FECHA_SINTOMAS ENTIDAD_RES    Grupo de Edad                   
2020-03-01     AGUASCALIENTES Menor de 2 años             0   
                              2 a 4 años                  0   
                              5 a 9 años                  0   
                              10 a 14 años        

In [107]:
agregados.reset_index().to_csv("data/agregados_infantil.csv", index=False)

In [22]:
df.SEXO

array(['CIUDAD DE MÉXICO'], dtype=object)